In [3]:
import xarray as xr
import pandas as pd
import plotly.express as px

In [4]:
barents = xr.load_dataset("../1_data_processing/1_3_Sensibility/barents.zarr", engine="zarr")
hot = xr.load_dataset("../1_data_processing/1_3_Sensibility/hot.zarr", engine="zarr")
# south_pole = xr.load_dataset("../1_data_processing/1_3_Sensibility/south_pole.zarr", engine="zarr")
# upwelling_south_america = xr.load_dataset(
#     "../1_data_processing/1_3_Sensibility/upwelling_south_america.zarr", engine="zarr"
# )
bats = xr.load_dataset("../1_data_processing/1_3_Sensibility/bats.zarr", engine="zarr")
papa = xr.load_dataset("../1_data_processing/1_3_Sensibility/papa.zarr", engine="zarr")
guam = xr.load_dataset("../1_data_processing/1_3_Sensibility/guam.zarr", engine="zarr")

stations = [barents, hot, bats, papa, guam]
names = ["BARENTS", "HOT", "BATS", "PAPA", "GUAM"]

Standardize format


In [5]:
table = []

table = [station.mean("time").to_dataframe().reset_index() for station in stations]
table = pd.concat(table).reset_index(drop=True)
table["name"] = names
table = table.rename(columns={"T": "temperature", "npp": "primary production"})
table

,longitude,latitude,temperature,primary production,name
0,26.969,74.620,4.036164,121.380569,BARENTS
1,-158.004,22.752,23.839729,254.277267,HOT
2,-64.200,31.604,21.537741,265.166229,BATS
3,-149.996,50.006,6.785365,276.715942,PAPA
4,149.995,13.001,27.390701,112.102440,GUAM


In [6]:
fig = px.scatter(table, x="temperature", y="primary production", color="name")
fig.update_layout(width=700, height=500)
fig.show()

Export in unique file


In [7]:
export_file = xr.merge(stations)
export_file

<xarray.Dataset> Size: 2MB
Dimensions:    (latitude: 5, longitude: 5, time: 9496)
Coordinates:
  * latitude   (latitude) float64 40B 13.0 22.75 31.6 50.01 74.62
  * longitude  (longitude) float64 40B -158.0 -150.0 -64.2 26.97 150.0
  * time       (time) datetime64[ns] 76kB 1998-01-01 1998-01-02 ... 2023-12-31
Data variables:
    T          (longitude, latitude, time) float32 950kB nan nan nan ... nan nan
    npp        (longitude, latitude, time) float32 950kB nan nan nan ... nan nan
Attributes:
    Conventions:       CF-1.11
    history:           Created on 2024-09-6
    institution:       CLS
    references:        http://www.cls.fr; http://www.seapodym.eu
    source:            SEAPODYM-LMTL 3.0.0
    subset:datasetId:  cmems_mod_glo_bgc_my_0.083deg-lmtl-Fphy_PT1D-i_202411
    subset:date:       2025-03-14T12:07:51.550Z
    subset:productId:  GLOBAL_MULTIYEAR_BGC_001_033
    subset:source:     ARCO data downloaded from the Marine Data Store using ...
    title:             Physical forcings of global ocean low and mid trophic ...

In [8]:
export_file["T"].attrs["units"] = "degree_Celsius"
export_file.latitude.attrs = {"axis": "Y"}
export_file.longitude.attrs = {"axis": "X"}
export_file["T"] = export_file["T"].expand_dims({"depth": [0]})
export_file["depth"].attrs["axis"] = "Z"
export_file = export_file.transpose("time", "latitude", "longitude", "depth")
export_file

<xarray.Dataset> Size: 2MB
Dimensions:    (latitude: 5, longitude: 5, time: 9496, depth: 1)
Coordinates:
  * latitude   (latitude) float64 40B 13.0 22.75 31.6 50.01 74.62
  * longitude  (longitude) float64 40B -158.0 -150.0 -64.2 26.97 150.0
  * time       (time) datetime64[ns] 76kB 1998-01-01 1998-01-02 ... 2023-12-31
  * depth      (depth) int64 8B 0
Data variables:
    T          (time, latitude, longitude, depth) float32 950kB nan nan ... nan
    npp        (time, latitude, longitude) float32 950kB nan nan ... nan
Attributes:
    Conventions:       CF-1.11
    history:           Created on 2024-09-6
    institution:       CLS
    references:        http://www.cls.fr; http://www.seapodym.eu
    source:            SEAPODYM-LMTL 3.0.0
    subset:datasetId:  cmems_mod_glo_bgc_my_0.083deg-lmtl-Fphy_PT1D-i_202411
    subset:date:       2025-03-14T12:07:51.550Z
    subset:productId:  GLOBAL_MULTIYEAR_BGC_001_033
    subset:source:     ARCO data downloaded from the Marine Data Store using ...
    title:             Physical forcings of global ocean low and mid trophic ...

In [9]:
export_file.to_zarr("../1_data_processing/1_3_Sensibility/all_stations.zarr", mode="w")

Export stations locations


In [11]:
table.to_json("../1_data_processing/1_3_Sensibility/stations_locations.json")